In [1]:
# Keras model 

In [2]:
import numpy as np 
import tensorflow as tf
import os 

In [3]:
ROOT_DIR = '/Users/joshua.newnham/Documents/Shared Playground Data/Sketches/preprocessed/'
VALID_DIR = os.path.join(ROOT_DIR, "valid")
TRAIN_DIR = os.path.join(ROOT_DIR, "train")
WEIGHTS_FILE = "sketch_classifier.h5"

TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 8

In [4]:
def count_files(full_path):
    count = 0 
    def is_image(file_path):
        image_extensions = ['png', 'jpg', 'jpeg']
        
        for image_extension in image_extensions:
            if image_extension in file_path.lower():
                return True
            
        return False 
    
    for d in os.listdir(full_path):
        if not os.path.isdir(os.path.join(full_path, d)):
            continue
            
        sub_full_path = os.path.join(full_path, d)
        
        for f in os.listdir(sub_full_path):
            img_path = os.path.join(sub_full_path, f)
            if os.path.isfile(img_path) and is_image(img_path):
                count += 1
            
    return count

In [5]:
NUM_TRAIN_FILES = count_files(TRAIN_DIR)
NUM_VALID_FILES = count_files(VALID_DIR)

In [6]:
def train(model):
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
    
    # create an iterator for the training data 
    train_generator = datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE, 
        color_mode='grayscale')
    
    # create an iterator for the validation data 
    validation_generator = datagen.flow_from_directory(
        VALID_DIR,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE, 
        color_mode='grayscale')
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(WEIGHTS_FILE, 
                                           monitor='val_loss', 
                                           verbose=0, 
                                           save_best_only=True, 
                                           save_weights_only=True, 
                                           mode='auto', 
                                           period=2)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=int(NUM_TRAIN_FILES/BATCH_SIZE),
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=int(NUM_VALID_FILES/BATCH_SIZE), 
        callbacks=[checkpoint, early_stopping]) 
    
    return history, model 

In [7]:
def validate_model(model):
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
    
    # create an iterator for the validation data 
    validation_generator = datagen.flow_from_directory(
        VALID_DIR,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE, 
        color_mode='grayscale')
    
    score = model.evaluate_generator(validation_generator)
    
    return score 

--- 

In [83]:
def create_model():
    """
    65% accuracy input size 68
    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3), 
                                     strides=(1,1),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE, 
                                     name="l1_conv"))    
    model.add(tf.keras.layers.Activation('relu', name="l1_relu"))    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3), 
                                     strides=(1,1),  
                                     padding='same',                                       
                                     activation=None, 
                                     name="l1_conv_b"))    
    model.add(tf.keras.layers.Activation('relu', name="l1_relu_b"))    
    model.add(tf.keras.layers.MaxPooling2D(3,3, name="l1_maxpool", padding='same'))    
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None, 
                                     name="l2_conv"))            
    model.add(tf.keras.layers.Activation('relu', name="l2_relu"))
    model.add(tf.keras.layers.MaxPooling2D(2,2, name="l2_maxpool", padding='same')) 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None, 
                                     name="l3_conv"))        
    model.add(tf.keras.layers.Activation('relu', name="l3_relu"))
    model.add(tf.keras.layers.MaxPooling2D(2,2, name="l3_maxpool", padding='same')) 
    model.add(tf.keras.layers.Dropout(0.4, name="l3_dropout"))
        
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(512, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Dense(256, 
                                    activation=None, 
                                    name='l4_fc'))    
    model.add(tf.keras.layers.Activation('relu', name="l4_relu"))
    model.add(tf.keras.layers.Dropout(0.4, name="l4_dropout"))
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, 
                                    activation=None, 
                                    name='l5_fc'))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [84]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
450/450 [==============================] - 25s 57ms/step - loss: 2.5642 - acc: 0.2110 - val_loss: 1.9611 - val_acc: 0.3864
Epoch 2/1000
450/450 [==============================] - 24s 54ms/step - loss: 2.0212 - acc: 0.3814 - val_loss: 1.8084 - val_acc: 0.4943
Epoch 3/1000
450/450 [==============================] - 23s 51ms/step - loss: 1.7381 - acc: 0.4638 - val_loss: 1.6036 - val_acc: 0.5057
Epoch 4/1000
450/450 [==============================] - 22s 48ms/step - loss: 1.5279 - acc: 0.5251 - val_loss: 1.5731 - val_acc: 0.5341
Epoch 5/1000
450/450 [==============================] - 25s 55ms/step - loss: 1.3486 - acc: 0.5730 - val_loss: 1.4252 - val_acc: 0.6023
Epoch 6/1000
450/450 [==============================] - 20s 45ms/step - loss: 1.2041 - acc: 0.6149 - val_loss: 1.4535 - val_acc: 0.6250
Epoch 7/1000
450/450 [==============================] - 26s 57ms/step - loss: 1.0633 - acc: 0.6630 

--- 

In [55]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
225/225 [==============================] - 78s 348ms/step - loss: 2.8129 - acc: 0.1676 - val_loss: 2.8365 - val_acc: 0.3636
Epoch 2/1000
225/225 [==============================] - 76s 340ms/step - loss: 2.2693 - acc: 0.3150 - val_loss: 2.1543 - val_acc: 0.4602
Epoch 3/1000
225/225 [==============================] - 79s 351ms/step - loss: 2.0154 - acc: 0.3829 - val_loss: 1.8296 - val_acc: 0.5114
Epoch 4/1000
225/225 [==============================] - 81s 360ms/step - loss: 1.7968 - acc: 0.4433 - val_loss: 1.5984 - val_acc: 0.5625
Epoch 5/1000
225/225 [==============================] - 82s 366ms/step - loss: 1.6842 - acc: 0.4804 - val_loss: 1.4696 - val_acc: 0.5909
Epoch 6/1000
225/225 [==============================] - 82s 363ms/step - loss: 1.5768 - acc: 0.5103 - val_loss: 1.4825 - val_acc: 0.5511
Epoch 7/1000
225/225 [==============================] - 80s 357ms/step - loss: 1.4380 - acc: 

--- 

In [56]:
def create_model():
    model = tf.keras.models.Sequential() 
    model.add(tf.keras.layers.Conv2D(16, kernel_size=(7,7), strides=(3,3),  
                            padding='same', activation='relu', input_shape=INPUT_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D(2,2, padding='same'))    
    
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(5,5), padding='same', activation='relu'))        
    model.add(tf.keras.layers.MaxPooling2D(2,2, padding='same'))    
    
    model.add(tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2,2, padding='same'))    
    model.add(tf.keras.layers.Dropout(0.125))
    
    model.add(tf.keras.layers.Conv2D(128, (5,5), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2,2))    
    
    model.add(tf.keras.layers.Flatten())                
    model.add(tf.keras.layers.Dense(256, activation='relu', name='dense_2_512'))
    model.add(tf.keras.layers.Dropout(0.3))
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name='output'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='rmsprop', 
        metrics=['accuracy'])
    
    return model 

In [57]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
225/225 [==============================] - 6s 26ms/step - loss: 2.8709 - acc: 0.1103 - val_loss: 2.4940 - val_acc: 0.1989
Epoch 2/1000
225/225 [==============================] - 5s 22ms/step - loss: 2.3486 - acc: 0.2643 - val_loss: 2.1257 - val_acc: 0.3239
Epoch 3/1000
225/225 [==============================] - 4s 20ms/step - loss: 2.0257 - acc: 0.3671 - val_loss: 2.0584 - val_acc: 0.3864
Epoch 4/1000
225/225 [==============================] - 4s 19ms/step - loss: 1.8254 - acc: 0.4257 - val_loss: 1.8259 - val_acc: 0.4943
Epoch 5/1000
225/225 [==============================] - 4s 20ms/step - loss: 1.6348 - acc: 0.4779 - val_loss: 1.7840 - val_acc: 0.4830
Epoch 6/1000
225/225 [==============================] - 4s 19ms/step - loss: 1.4563 - acc: 0.5352 - val_loss: 1.7792 - val_acc: 0.4773
Epoch 7/1000
225/225 [==============================] - 4s 19ms/step - loss: 1.3071 - acc: 0.5807 - val_l

--- 

In [58]:
def create_model():
    model = tf.keras.models.Sequential() 
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), strides=(3,3),  
                            padding='same', activation='relu', input_shape=INPUT_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D(2,2))    
    
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu'))        
    model.add(tf.keras.layers.MaxPooling2D(2,2))      
    
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu'))        
    model.add(tf.keras.layers.MaxPooling2D(2,2))   
    model.add(tf.keras.layers.Dropout(0.33))    
    
    model.add(tf.keras.layers.Flatten())                
    
    model.add(tf.keras.layers.Dense(256, activation='relu', name='dense_2_512'))
    model.add(tf.keras.layers.Dropout(0.33))
    
#     model.add(tf.keras.layers.Dense(256, activation='relu'))
#     model.add(tf.keras.layers.Dropout(0.33))
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name='output'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='rmsprop', 
        metrics=['accuracy'])
    
    return model 

In [59]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
225/225 [==============================] - 4s 19ms/step - loss: 2.9253 - acc: 0.1126 - val_loss: 2.4768 - val_acc: 0.2500
Epoch 2/1000
225/225 [==============================] - 4s 16ms/step - loss: 2.4946 - acc: 0.2264 - val_loss: 2.1238 - val_acc: 0.3750
Epoch 3/1000
225/225 [==============================] - 3s 15ms/step - loss: 2.2695 - acc: 0.2923 - val_loss: 1.9798 - val_acc: 0.3580
Epoch 4/1000
225/225 [==============================] - 3s 15ms/step - loss: 2.1537 - acc: 0.3220 - val_loss: 1.9027 - val_acc: 0.4545
Epoch 5/1000
225/225 [==============================] - 3s 15ms/step - loss: 2.0305 - acc: 0.3713 - val_loss: 1.8102 - val_acc: 0.4261
Epoch 6/1000
225/225 [==============================] - 3s 14ms/step - loss: 1.9485 - acc: 0.3893 - val_loss: 1.7647 - val_acc: 0.4432
Epoch 7/1000
225/225 [==============================] - 3s 15ms/step - loss: 1.8978 - acc: 0.4000 - val_l

--- 

In [60]:
def create_model():
    model = tf.keras.models.Sequential() 
    model.add(tf.keras.layers.Conv2D(16, kernel_size=(5,5), strides=(3,3),  
                            padding='same', activation='relu', input_shape=INPUT_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D(2,2, padding='same'))    
    
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu'))        
    model.add(tf.keras.layers.MaxPooling2D(2,2, padding='same'))   
    model.add(tf.keras.layers.Dropout(0.33))    
    
    model.add(tf.keras.layers.Flatten())                
    
    model.add(tf.keras.layers.Dense(256, activation='relu', name='dense_2_512'))
    model.add(tf.keras.layers.Dropout(0.33))
    
#     model.add(tf.keras.layers.Dense(256, activation='relu'))
#     model.add(tf.keras.layers.Dropout(0.33))
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name='output'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='rmsprop', 
        metrics=['accuracy'])
    
    return model 

In [61]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
225/225 [==============================] - 4s 20ms/step - loss: 2.6679 - acc: 0.1829 - val_loss: 2.1610 - val_acc: 0.3239
Epoch 2/1000
225/225 [==============================] - 4s 17ms/step - loss: 2.2071 - acc: 0.3184 - val_loss: 1.8687 - val_acc: 0.4602
Epoch 3/1000
225/225 [==============================] - 3s 15ms/step - loss: 1.9675 - acc: 0.3945 - val_loss: 1.7776 - val_acc: 0.4659
Epoch 4/1000
225/225 [==============================] - 3s 15ms/step - loss: 1.8038 - acc: 0.4513 - val_loss: 1.7342 - val_acc: 0.4602
Epoch 5/1000
225/225 [==============================] - 3s 14ms/step - loss: 1.6667 - acc: 0.4898 - val_loss: 1.6631 - val_acc: 0.5114
Epoch 6/1000
225/225 [==============================] - 3s 15ms/step - loss: 1.5413 - acc: 0.5140 - val_loss: 1.6376 - val_acc: 0.5057
Epoch 7/1000
225/225 [==============================] - 3s 15ms/step - loss: 1.4631 - acc: 0.5408 - val_l

--- 

In [44]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
    
# create an iterator for the training data 
train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=TARGET_SIZE,
    batch_size=NUM_TRAIN_FILES, 
    color_mode='grayscale')

# create an iterator for the validation data 
validation_generator = datagen.flow_from_directory(
    VALID_DIR,
    target_size=TARGET_SIZE,
    batch_size=NUM_VALID_FILES, 
    color_mode='grayscale')

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.


In [45]:
x_train, y_train = train_generator.next()
x_test, y_test = validation_generator.next()

In [46]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 99.67%


In [47]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 65.91%


## Export weights 

Export weights
The type of each entry in array is given by -dataType. The number of entries is equal to:

inputFeatureChannels outputFeatureChannels kernelHeight kernelWidth*

The layout of filter weight is as a 4D tensor (array) weight[ outputChannels ][ kernelHeight ][ kernelWidth ][ inputChannels / groups ]

Note: For binary-convolutions the layout of the weights are: weight[ outputChannels ][ kernelHeight ][ kernelWidth ][ floor((inputChannels/groups)+31) / 32 ] with each 32 sub input feature channel index specified in machine byte order, so that for example the 13th feature channel bit can be extracted using bitmask = (1U << 13).

In [ ]:
def export_conv_weights(name, wts_coef, bias_coef):
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv.data".format(name)), 
          os.path.join('exports', "{}_bias.data".format(name))))
    
    print("\n")
    
    # [kernel_width, kernel_height, input_feature_channels, output_feature_channels]
    print("\tOriginal weights shape {}".format(wts_coef.shape))
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "{}_conv.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "{}_bias.data".format(name)))    
        
    print("\n")

In [ ]:
def export_dense_weights(name, wts_coef, bias_coef, kernel_width, kernel_height):
    """
    A fully connected layer in a Convolutional Neural Network (CNN) is one where every input channel is connected 
    to every output channel. The kernel width is equal to the width of the source image, and the 
    kernel height is equal to the height of the source image. The width and height of the output is 1 x 1.
    
    A fully connected layer takes an MPSImage object with dimensions 
    source.width x source.height x Ni, convolves it with Weights[No][source.width][source.height][Ni], 
    and produces a 1 x 1 x No output.
    
    Thus, the following conditions must be true:
    - kernelWidth == source.width
    - kernelHeight == source.height
    - clipRect.size.width == 1
    - clipRect.size.height == 1
    
    You can think of a fully connected layer as a matrix multiplication where the image is 
    flattened into a vector of length source.width*source.height*Ni, and the weights are arranged in a 
    matrix of dimension No x (source.width*source.height*Ni) to produce an output vector of length No.
    
    The value of the strideInPixelsX, strideInPixelsY, and groups properties must be 1. 
    The offset property is not applicable and it is ignored. Because the clip rectangle is 
    clamped to the destination image bounds, if the destination is 1 x 1, you do not need to set the 
    clipRect property.
    """
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv.data".format(name)), 
          os.path.join('exports', "{}_bias.data".format(name))))
        
    input_feature_channels = int(wts_coef.shape[0] / kernel_width / kernel_height) 
    output_feature_channels = wts_coef.shape[-1]            
    
    # [kernel_width, kernel_height, input_feature_channels, output_feature_channels]
    print("\tOriginal weights shape {}".format(wts_coef.shape))
    
    #wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, input_feature_channels, output_feature_channels])    
    wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, -1, output_feature_channels])    
        
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "{}_conv.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "{}_bias.data".format(name)))   
        
    print("\n")

In [ ]:
flatted_input_kernel_width = None
flatted_input_kernel_height = None

for layer in model.layers:        
    if "flatten" in layer.name:
        flatted_input_kernel_width = layer.input_shape[1] # None, 14, 14, 64
        flatted_input_kernel_height = layer.input_shape[2] # None, 14, 14, 64
        print("flatted_input_kernel_width {} flatted_input_kernel_height {}".format(
            flatted_input_kernel_width, flatted_input_kernel_height))
        
    if len(layer.get_weights()) > 0:        
        name = layer.name         
        wts = layer.get_weights()
        
        if "conv" in name:            
            export_conv_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None)        
        else:
            export_dense_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None, 
                                flatted_input_kernel_width, flatted_input_kernel_height)        
            # after the initial pass (from cnn to fcn); flattern the kernel down to 1x1 
            # i.e. update the flatted_input_kernel_DIM to have the kernel width and height of 1 
            flatted_input_kernel_width, flatted_input_kernel_height = 1, 1             

--- 

In [29]:
def create_model():
    """
    https://www.iioab.org/articles/IIOABJ_7.S5_337-341.pdf
    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(96, 
                                     kernel_size=(10,10), 
                                     strides=(1,1),  
                                     padding='valid',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
    
    model.add(tf.keras.layers.Conv2D(192, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='valid', 
                                     activation=None))            
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
    model.add(tf.keras.layers.Dropout(0.35))
    
    model.add(tf.keras.layers.Conv2D(192, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='valid', 
                                     activation=None))            
    model.add(tf.keras.layers.Activation('relu'))
        
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(192, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.35))
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES,activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer="sgd", # tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [30]:
model = create_model() 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 55, 55, 96)        9696      
_________________________________________________________________
activation_20 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 23, 23, 192)       460992    
_________________________________________________________________
activation_21 (Activation)   (None, 23, 23, 192)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 11, 11, 192)       0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 11, 11, 192)       0         
__________

In [31]:
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
225/225 [==============================] - 53s 235ms/step - loss: 3.0778 - acc: 0.0553 - val_loss: 3.0429 - val_acc: 0.1364
Epoch 2/1000
225/225 [==============================] - 48s 215ms/step - loss: 2.9464 - acc: 0.1145 - val_loss: 2.6928 - val_acc: 0.1989
Epoch 3/1000
225/225 [==============================] - 49s 219ms/step - loss: 2.6573 - acc: 0.2001 - val_loss: 2.4005 - val_acc: 0.3011
Epoch 4/1000
225/225 [==============================] - 49s 219ms/step - loss: 2.4546 - acc: 0.2484 - val_loss: 2.1870 - val_acc: 0.3693
Epoch 5/1000
225/225 [==============================] - 48s 212ms/step - loss: 2.3132 - acc: 0.2868 - val_loss: 2.0689 - val_acc: 0.3920
Epoch 6/1000
225/225 [==============================] - 48s 211ms/step - loss: 2.2147 - acc: 0.3164 - val_loss: 1.9824 - val_acc: 0.4148
Epoch 7/1000
225/225 [==============================] - 49s 218ms/step - loss: 2.1061 - acc: 

--- 

In [54]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(3,3))
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))        
    model.add(tf.keras.layers.Dropout(0.4)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(32, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [55]:
model = create_model() 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 64, 64, 32)        1600      
_________________________________________________________________
activation_81 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_63 (Dropout)         (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 64, 64, 32)        25632     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 21, 21, 32)        0         
_________________________________________________________________
activation_82 (Activation)   (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 21, 21, 32)        25632     
__________

In [56]:
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
450/450 [==============================] - 60s 133ms/step - loss: 2.9245 - acc: 0.1098 - val_loss: 2.4482 - val_acc: 0.2784
Epoch 2/1000
450/450 [==============================] - 59s 131ms/step - loss: 2.4531 - acc: 0.2420 - val_loss: 2.1753 - val_acc: 0.4034
Epoch 3/1000
450/450 [==============================] - 57s 126ms/step - loss: 2.2052 - acc: 0.3226 - val_loss: 1.9569 - val_acc: 0.4716
Epoch 4/1000
450/450 [==============================] - 57s 128ms/step - loss: 2.0291 - acc: 0.3707 - val_loss: 1.6818 - val_acc: 0.5284
Epoch 5/1000
450/450 [==============================] - 57s 127ms/step - loss: 1.8622 - acc: 0.4229 - val_loss: 1.6938 - val_acc: 0.5227
Epoch 6/1000
450/450 [==============================] - 60s 133ms/step - loss: 1.7634 - acc: 0.4637 - val_loss: 1.6022 - val_acc: 0.5739
Epoch 7/1000
450/450 [==============================] - 58s 130ms/step - loss: 1.6562 - acc: 

---

In [88]:
TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 64

In [89]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))        
    model.add(tf.keras.layers.Dropout(0.4)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(32, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [90]:
model = create_model()
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
56/56 [==============================] - 53s 948ms/step - loss: 2.9346 - acc: 0.1220 - val_loss: 2.6917 - val_acc: 0.2500
Epoch 2/1000
56/56 [==============================] - 52s 929ms/step - loss: 2.5938 - acc: 0.2317 - val_loss: 2.2908 - val_acc: 0.3516
Epoch 3/1000
56/56 [==============================] - 51s 915ms/step - loss: 2.3669 - acc: 0.2942 - val_loss: 2.1253 - val_acc: 0.4609
Epoch 4/1000
56/56 [==============================] - 51s 919ms/step - loss: 2.1921 - acc: 0.3366 - val_loss: 1.9819 - val_acc: 0.4688
Epoch 5/1000
56/56 [==============================] - 51s 907ms/step - loss: 2.0813 - acc: 0.3659 - val_loss: 1.7306 - val_acc: 0.5312
Epoch 6/1000
56/56 [==============================] - 51s 919ms/step - loss: 1.9778 - acc: 0.4036 - val_loss: 1.5973 - val_acc: 0.5391
Epoch 7/1000
56/56 [==============================] - 51s 906ms/step - loss: 1.8926 - acc: 0.4161 - val_l

--- 

In [67]:
TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 32

In [75]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu')) 
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.Activation('relu'))    
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.Activation('relu'))
              
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Dropout(0.25)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(32, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.25))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [76]:
model = create_model()
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
112/112 [==============================] - 127s 1s/step - loss: 3.0368 - acc: 0.0882 - val_loss: 2.6109 - val_acc: 0.2313
Epoch 2/1000
112/112 [==============================] - 127s 1s/step - loss: 2.5311 - acc: 0.2525 - val_loss: 2.0425 - val_acc: 0.4125
Epoch 3/1000
112/112 [==============================] - 125s 1s/step - loss: 2.1095 - acc: 0.3657 - val_loss: 1.7335 - val_acc: 0.4813
Epoch 4/1000
112/112 [==============================] - 125s 1s/step - loss: 1.8208 - acc: 0.4592 - val_loss: 1.5732 - val_acc: 0.5188
Epoch 5/1000
112/112 [==============================] - 121s 1s/step - loss: 1.5987 - acc: 0.5054 - val_loss: 1.5318 - val_acc: 0.5687
Epoch 6/1000
112/112 [==============================] - 121s 1s/step - loss: 1.4283 - acc: 0.5574 - val_loss: 1.4741 - val_acc: 0.5625
Epoch 7/1000
112/112 [==============================] - 121s 1s/step - loss: 1.2236 - acc: 0.6106 - val_l

--- 

In [82]:
TARGET_SIZE = (32,32)
INPUT_SHAPE = (32,32,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 32

In [83]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3), 
                                     strides=(1,1),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Dropout(0.2))
    
    model.add(tf.keras.layers.Conv2D(64, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Conv2D(64, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(512, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [84]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
112/112 [==============================] - 10s 91ms/step - loss: 2.4908 - acc: 0.2506 - val_loss: 2.0344 - val_acc: 0.3563
Epoch 2/1000
112/112 [==============================] - 9s 78ms/step - loss: 1.9799 - acc: 0.3890 - val_loss: 1.8709 - val_acc: 0.4437
Epoch 3/1000
112/112 [==============================] - 8s 72ms/step - loss: 1.7059 - acc: 0.4682 - val_loss: 1.8202 - val_acc: 0.4562
Epoch 4/1000
112/112 [==============================] - 8s 71ms/step - loss: 1.4090 - acc: 0.5593 - val_loss: 2.0390 - val_acc: 0.4500
Epoch 5/1000
112/112 [==============================] - 8s 72ms/step - loss: 1.1566 - acc: 0.6319 - val_loss: 1.8169 - val_acc: 0.5000
Epoch 6/1000
112/112 [==============================] - 8s 72ms/step - loss: 0.8283 - acc: 0.7335 - val_loss: 1.9625 - val_acc: 0.4750
Epoch 7/1000
112/112 [==============================] - 8s 73ms/step - loss: 0.5617 - acc: 0.8252 - val_

--- 

In [1]:
TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 32

In [2]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))        
    model.add(tf.keras.layers.Dropout(0.3)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(64, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        #optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        optimizer=tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [110]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
112/112 [==============================] - 64s 570ms/step - loss: 2.8324 - acc: 0.1256 - val_loss: 2.3976 - val_acc: 0.3312
Epoch 2/1000
112/112 [==============================] - 63s 560ms/step - loss: 2.3354 - acc: 0.2937 - val_loss: 1.9252 - val_acc: 0.4875
Epoch 3/1000
112/112 [==============================] - 59s 523ms/step - loss: 1.9952 - acc: 0.3981 - val_loss: 1.7035 - val_acc: 0.5500
Epoch 4/1000
112/112 [==============================] - 62s 555ms/step - loss: 1.7516 - acc: 0.4651 - val_loss: 1.5771 - val_acc: 0.5563
Epoch 5/1000
112/112 [==============================] - 60s 536ms/step - loss: 1.6271 - acc: 0.5090 - val_loss: 1.4593 - val_acc: 0.5813
Epoch 6/1000
112/112 [==============================] - 59s 531ms/step - loss: 1.4718 - acc: 0.5449 - val_loss: 1.4329 - val_acc: 0.5563
Epoch 7/1000
112/112 [==============================] - 58s 521ms/step - loss: 1.3701 - acc: 

--- 

In [13]:
TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 32

In [19]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(3,3))
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(3,3),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))        
    model.add(tf.keras.layers.Dropout(0.3)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(64, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        #optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        optimizer=tf.keras.optimizers.Adam(), 
#         optimizer=tf.keras.optimizers.SGD(lr=0.01),
        metrics=['accuracy']) 
    
    return model 

In [ ]:
model = create_model() 
history, model = train(model)

--- 

In [9]:
TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 66

In [10]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(3,3))
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))        
    model.add(tf.keras.layers.Dropout(0.3)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(64, activation=None))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        #optimizer="rmsprop", # tf.keras.optimizers.Adam(), 
        optimizer=tf.keras.optimizers.Adam(), 
        metrics=['accuracy']) 
    
    return model 

In [11]:
model = create_model() 
history, model = train(model)

Found 3605 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
54/54 [==============================] - 42s 782ms/step - loss: 2.9771 - acc: 0.0895 - val_loss: 2.5263 - val_acc: 0.2424
Epoch 2/1000
54/54 [==============================] - 41s 759ms/step - loss: 2.5481 - acc: 0.2166 - val_loss: 2.0953 - val_acc: 0.3939
Epoch 3/1000
54/54 [==============================] - 41s 760ms/step - loss: 2.2701 - acc: 0.3002 - val_loss: 1.8846 - val_acc: 0.4621
Epoch 4/1000
54/54 [==============================] - 41s 758ms/step - loss: 2.0833 - acc: 0.3640 - val_loss: 1.8568 - val_acc: 0.4545
Epoch 5/1000
54/54 [==============================] - 40s 735ms/step - loss: 1.9221 - acc: 0.4128 - val_loss: 1.6674 - val_acc: 0.5379
Epoch 6/1000
54/54 [==============================] - 41s 759ms/step - loss: 1.8353 - acc: 0.4408 - val_loss: 1.6373 - val_acc: 0.4924
Epoch 7/1000
54/54 [==============================] - 41s 755ms/step - loss: 1.7152 - acc: 0.4780 - val_l

--- 

In [8]:
TARGET_SIZE = (128,128)
INPUT_SHAPE = (128,128,1)
NUM_CLASSES = 22

EPOCHS = 1000 
BATCH_SIZE = 4

In [9]:
def create_model():
    """

    """
    model = tf.keras.models.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(7,7), 
                                     strides=(2,2),  
                                     padding='same',                                       
                                     activation=None, 
                                     input_shape=INPUT_SHAPE, 
                                     name='l1'))    
    model.add(tf.keras.layers.Activation('relu'))           
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None, 
                                     name='l2'))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None, 
                                     name='l3'))            
    model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(32, 
                                     kernel_size=(5,5),
                                     strides=(1,1),  
                                     padding='same', 
                                     activation=None, 
                                     name='l4'))        
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPool2D(2,2))        
    model.add(tf.keras.layers.Dropout(0.3)) 
    
    model.add(tf.keras.layers.Flatten())      
    
    model.add(tf.keras.layers.Dense(64, activation=None, name='l5'))    
    model.add(tf.keras.layers.Activation('relu'))    
    model.add(tf.keras.layers.Dropout(0.3))    
        
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=None, name='l6'))
    model.add(tf.keras.layers.Activation('softmax', name="output"))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=tf.keras.optimizers.SGD(lr=0.01), 
        metrics=['accuracy']) 
    
    return model 

In [10]:
model = create_model() 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
l1 (Conv2D)                  (None, 64, 64, 32)        1600      
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
l2 (Conv2D)                  (None, 64, 64, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
l3 (Conv2D)                  (None, 32, 32, 32)        25632     
__________

In [11]:
history, model = train(model)

Found 2844 images belonging to 22 classes.
Found 176 images belonging to 22 classes.
Epoch 1/1000
711/711 [==============================] - 44s 62ms/step - loss: 3.0878 - acc: 0.0489 - val_loss: 3.0585 - val_acc: 0.1023
Epoch 2/1000
711/711 [==============================] - 43s 61ms/step - loss: 2.8799 - acc: 0.1213 - val_loss: 2.5609 - val_acc: 0.2557
Epoch 3/1000
711/711 [==============================] - 44s 62ms/step - loss: 2.5441 - acc: 0.2201 - val_loss: 2.2478 - val_acc: 0.3523
Epoch 4/1000
711/711 [==============================] - 35s 49ms/step - loss: 2.2611 - acc: 0.3119 - val_loss: 1.9418 - val_acc: 0.4489
Epoch 5/1000
711/711 [==============================] - 33s 47ms/step - loss: 2.0837 - acc: 0.3625 - val_loss: 1.7817 - val_acc: 0.5114
Epoch 6/1000
711/711 [==============================] - 34s 47ms/step - loss: 1.9059 - acc: 0.4153 - val_loss: 1.7432 - val_acc: 0.5114
Epoch 7/1000
711/711 [==============================] - 33s 47ms/step - loss: 1.7598 - acc: 0.4508 

In [12]:
score = validate_model(model)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Found 176 images belonging to 22 classes.
('Test loss:', 1.2765541143122723)
('Test accuracy:', 0.6363636363636364)


In [13]:
import json 
from keras.models import model_from_json

with open('sketch_cnn.json', 'w') as f:
    json_obj = json.loads(model.to_json())
    json.dump(json_obj, f)

model.save_weights('sketch_cnn.h5')

Using TensorFlow backend.


## Export Weights 

The type of each entry in array is given by -dataType. The number of entries is equal to:

inputFeatureChannels outputFeatureChannels kernelHeight kernelWidth*

The layout of filter weight is as a 4D tensor (array) weight[ outputChannels ][ kernelHeight ][ kernelWidth ][ inputChannels / groups ]

Note: For binary-convolutions the layout of the weights are: weight[ outputChannels ][ kernelHeight ][ kernelWidth ][ floor((inputChannels/groups)+31) / 32 ] with each 32 sub input feature channel index specified in machine byte order, so that for example the 13th feature channel bit can be extracted using bitmask = (1U << 13).

In [14]:
def export_conv_weights(name, wts_coef, bias_coef):
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv.data".format(name)), 
          os.path.join('exports', "{}_bias.data".format(name))))
    
    print("\n")
    
    # [kernel_width, kernel_height, input_feature_channels, output_feature_channels]
    print("\tOriginal weights shape {}".format(wts_coef.shape))
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "{}_conv.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "{}_bias.data".format(name)))    
        
    print("\n")

In [15]:
def export_dense_weights(name, wts_coef, bias_coef, kernel_width, kernel_height):
    """
    A fully connected layer in a Convolutional Neural Network (CNN) is one where every input channel is connected 
    to every output channel. The kernel width is equal to the width of the source image, and the 
    kernel height is equal to the height of the source image. The width and height of the output is 1 x 1.
    
    A fully connected layer takes an MPSImage object with dimensions 
    source.width x source.height x Ni, convolves it with Weights[No][source.width][source.height][Ni], 
    and produces a 1 x 1 x No output.
    
    Thus, the following conditions must be true:
    - kernelWidth == source.width
    - kernelHeight == source.height
    - clipRect.size.width == 1
    - clipRect.size.height == 1
    
    You can think of a fully connected layer as a matrix multiplication where the image is 
    flattened into a vector of length source.width*source.height*Ni, and the weights are arranged in a 
    matrix of dimension No x (source.width*source.height*Ni) to produce an output vector of length No.
    
    The value of the strideInPixelsX, strideInPixelsY, and groups properties must be 1. 
    The offset property is not applicable and it is ignored. Because the clip rectangle is 
    clamped to the destination image bounds, if the destination is 1 x 1, you do not need to set the 
    clipRect property.
    """
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv.data".format(name)), 
          os.path.join('exports', "{}_bias.data".format(name))))
        
    input_feature_channels = int(wts_coef.shape[0] / kernel_width / kernel_height) 
    output_feature_channels = wts_coef.shape[-1]            
    
    # [kernel_width, kernel_height, input_feature_channels, output_feature_channels]
    print("\tOriginal weights shape {}".format(wts_coef.shape))
    
    #wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, input_feature_channels, output_feature_channels])    
    wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, -1, output_feature_channels])    
        
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "{}_conv.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "{}_bias.data".format(name)))   
        
    print("\n")

In [16]:
###### flatted_input_kernel_width = None
flatted_input_kernel_height = None

for layer in model.layers:        
    if "flatten" in layer.name:
        flatted_input_kernel_width = layer.input_shape[1] 
        flatted_input_kernel_height = layer.input_shape[2] 
        
    if len(layer.get_weights()) > 0:        
        name = layer.name
        wts = layer.get_weights()
        
        if name in ['l1', 'l2', 'l3', 'l4']:
            export_conv_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None)        
        elif name in ['l5', 'l6']:
            export_dense_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None, 
                                flatted_input_kernel_width, flatted_input_kernel_height)        
            # after the initial pass (from cnn to fcn); flattern the kernel down to 1x1 
            # i.e. update the flatted_input_kernel_DIM to have the kernel width and height of 1 
            flatted_input_kernel_width, flatted_input_kernel_height = 1, 1 

Exporting weights for l1
	exports/l1_conv.data
	exports/l1_bias.data


	Original weights shape (7, 7, 1, 32)
	Original bias shape (32,)
	Reshaped weights shape (32, 7, 7, 1)
	Reshaped bias_coef shape (32,)


Exporting weights for l2
	exports/l2_conv.data
	exports/l2_bias.data


	Original weights shape (5, 5, 32, 32)
	Original bias shape (32,)
	Reshaped weights shape (32, 5, 5, 32)
	Reshaped bias_coef shape (32,)


Exporting weights for l3
	exports/l3_conv.data
	exports/l3_bias.data


	Original weights shape (5, 5, 32, 32)
	Original bias shape (32,)
	Reshaped weights shape (32, 5, 5, 32)
	Reshaped bias_coef shape (32,)


Exporting weights for l4
	exports/l4_conv.data
	exports/l4_bias.data


	Original weights shape (5, 5, 32, 32)
	Original bias shape (32,)
	Reshaped weights shape (32, 5, 5, 32)
	Reshaped bias_coef shape (32,)


Exporting weights for l5
	exports/l5_conv.data
	exports/l5_bias.data
	Original weights shape (2048, 64)
	Original bias shape (64,)
	Reshaped weights shape (64, 8,

## Inspect models initlised weights 

In [17]:
tmp_model = create_model()

In [24]:
wts = tmp_model.layers[0].get_weights()

In [25]:
wts[0]

array([[[[ 0.05900718,  0.04470242, -0.04612648, ...,  0.04839333,
           0.00087062, -0.02341526]],

        [[-0.0533563 , -0.0567241 , -0.00595827, ...,  0.03955685,
          -0.00030188,  0.0498188 ]],

        [[ 0.01904735,  0.0105496 , -0.01652239, ...,  0.04525235,
          -0.03117971, -0.02302255]],

        ...,

        [[-0.0604782 ,  0.03106899, -0.03467188, ..., -0.04902239,
          -0.04092151, -0.03075201]],

        [[ 0.05375518,  0.0057967 , -0.03389849, ..., -0.04101713,
           0.02911748,  0.00312194]],

        [[-0.0527735 ,  0.01116358, -0.05607419, ..., -0.03250823,
          -0.03193898, -0.05470339]]],


       [[[ 0.06032657, -0.04008624, -0.0504399 , ...,  0.01788989,
          -0.00406332, -0.04738389]],

        [[-0.00438863,  0.02607093, -0.04305125, ...,  0.03807356,
          -0.00495933, -0.05969848]],

        [[-0.04564478, -0.0590722 , -0.02759542, ...,  0.05074384,
           0.00731297,  0.02065421]],

        ...,

        [[ 0.049

In [26]:
wts[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)